# Xgboost

### 載入套件

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

### 修改當下檔案開啟目錄

In [3]:
os.chdir("/Users/liouscott/Documents/scott/machine learning/Machine Learning A-Z Template Folder/Part 10 - Model Selection & Boosting/Section 49 - XGBoost/XGBoost/")

### 載入資料

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

### 將類別變數轉換為OneHotEncoder

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

### 切割train data和test data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Xgboost Model

Parameter | Description
:--- | :---
max_depth=3 | 和GBM中的參數相同，這個值為樹的最大深度。這個值也是用來避免過擬合的。max_depth越大，模型會學到更具體更局部的樣本。需要使用CV函數來進行調優。典型值：3-10
 learning_rate=0.1 |通過減少每一步的權重，可以提高模型的魯棒性。典型值為0.01-0.2。
  objective='binary:logistic'|這個參數定義需要被最小化的損失函數。最常用的值有： binary:logistic 二分類的邏輯迴歸，返回預測的概率(不是類別)。multi:softmax 使用softmax的多分類器，返回預測的類別(不是概率)。在這種情況下，你還需要多設一個參數：num_class(類別數目)。multi:softprob 和multi:softmax參數一樣，但是返回的是每個數據屬於各個類別的概率。
 booster='gbtree'|選擇每次迭代的模型，有兩種選擇： gbtree：基於樹的模型；gbliner：線性模型
  gamma=0|在節點分裂時，只有分裂後損失函數的值下降了，才會分裂這個節點。Gamma指定了節點分裂所需的最小損失函數下降值。這個參數的值越大，算法越保守。這個參數的值和損失函數息息相關，所以是需要調整的。"
   min_child_weight=1|"決定最小葉子節點樣本權重和。和GBM的 min_child_leaf 參數類似，但不完全一樣。XGBoost的這個參數是最小樣本權重的和，而GBM參數是最小樣本總數。這個參數用於避免過擬合。當它的值較大時，可以避免模型學習到局部的特殊樣本。但是如果這個值過高，會導致欠擬合。這個參數需要使用CV來調整。"
 max_delta_step=0|這參數限制每棵樹權重改變的最大步長。如果這個參數的值為0，那就意味著沒有約束。如果它被賦予了某個正值，那麼它會讓這個算法更加保守。通常，這個參數不需要設置。但是當各類別的樣本十分不平衡時，它對邏輯迴歸是很有幫助的。"
  reg_alpha=0|權重的L1正則化項。(和Lasso regression類似)。可以應用在很高維度的情況下，使得算法的速度更快。
 reg_lambda=1|權重的L2正則化項。(和Ridge regression類似)。這個參數是用來控制XGBoost的正則化部分的。雖然大部分數據科學家很少用到這個參數，但是這個參數在減少過擬合上還是可以挖掘出更多用處的。
 scale_pos_weight=1|在各類別樣本十分不平衡時，把這個參數設定為一個正值，可以使算法更快收斂。

參考網址：
1. https://weiwenku.net/d/100702111 (XGBoost參數調優完全指南)
2. http://xgboost.readthedocs.io/en/latest/python/python_api.html (XGBoost Api文件)

In [7]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

### 將test data放入估計完成的 XGBoost Model

In [8]:
y_pred = classifier.predict(X_test)

### 使用混淆矩陣檢視預測力

In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)

[[1521   74]
 [ 197  208]]


### 使用Cross Validation觀看精確的變異是否波動太大

In [19]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print ("mean:" + str(accuracies.mean()))
print ("std:" + str(accuracies.std()))

mean:0.862999445116
std:0.0106778721717
